In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from google.colab import drive

# Monta o Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class GuaraniTranslator:
    def __init__(self, model_path="/content/drive/MyDrive/Doutorado Unesp/assistente-guarani/data/modelo_finetunado"):
        """Inicializa o tradutor com modelo treinado"""
        print("🔄 Carregando modelo treinado...")

        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None
        )

        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"✅ Modelo carregado no device: {self.device}")

    def translate_to_guarani(self, texto_portugues):
        """Traduz do português para Guarani"""
        prompt = f"Instrução: Como se diz '{texto_portugues}' em Guarani?\nResposta:"
        return self._generate_response(prompt)

    def translate_to_portuguese(self, texto_guarani):
        """Traduz do Guarani para português"""
        prompt = f"Instrução: Traduza para o português:\nEntrada: {texto_guarani}\nResposta:"
        return self._generate_response(prompt)

    def _generate_response(self, prompt):
        """Gera resposta usando o modelo"""
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=256
        )

        if torch.cuda.is_available():
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=50,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )

        # Decodifica apenas os tokens novos
        response = self.tokenizer.decode(
            outputs[0][inputs["input_ids"].shape[1]:],
            skip_special_tokens=True
        )

        return response.strip()

In [4]:
def test_model():
    """Testa o modelo com frases de exemplo"""
    translator = GuaraniTranslator()

    # Frases de teste
    test_phrases_pt = [
        "bom dia",
        "como vai você",
        "eu te amo",
        "obrigado",
        "casa bonita"
    ]

    test_phrases_guarani = [
        "Aiko porã",
        "Aguyje",
        "Sy",
        "Kuarahy"
    ]

    print("\n🌅 TESTANDO PORTUGUÊS -> GUARANI:")
    print("=" * 50)
    for phrase in test_phrases_pt:
        result = translator.translate_to_guarani(phrase)
        print(f"'{phrase}' → '{result}'")

    print("\n🌄 TESTANDO GUARANI -> PORTUGUÊS:")
    print("=" * 50)
    for phrase in test_phrases_guarani:
        result = translator.translate_to_portuguese(phrase)
        print(f"'{phrase}' → '{result}'")

def interactive_mode():
    """Modo interativo para testar traduções"""
    translator = GuaraniTranslator()

    print("\n🎯 MODO INTERATIVO")
    print("Digite 'sair' para terminar")
    print("Digite 'pt:' + frase para traduzir para Guarani")
    print("Digite 'gn:' + frase para traduzir para Português")
    print("=" * 50)

    while True:
        user_input = input("\n💬 Digite sua frase: ").strip()

        if user_input.lower() == 'sair':
            print("👋 Até logo!")
            break

        if user_input.startswith('pt:'):
            texto = user_input[3:].strip()
            result = translator.translate_to_guarani(texto)
            print(f"🇧🇷 → 🇵🇾: '{result}'")

        elif user_input.startswith('gn:'):
            texto = user_input[3:].strip()
            result = translator.translate_to_portuguese(texto)
            print(f"🇵🇾 → 🇧🇷: '{result}'")

        else:
            print("⚠️ Use o formato: 'pt:frase' ou 'gn:frase'")



In [ ]:
# Executa testes e oferece modo interativo
test_model()

response = input("\n❓ Quer testar no modo interativo? (s/n): ")
if response.lower() in ['s', 'sim', 'y', 'yes']:
    interactive_mode()

🔄 Carregando modelo treinado...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Modelo carregado no device: cuda

🌅 TESTANDO PORTUGUÊS -> GUARANI:


/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0625 23:53:19.711000 683 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


'bom dia' → 'Vakuéra peteĩ tetãre rehegua.'
'como vai você' → 'Ko’ágã pe’ẽvape guarã ha’e: Ko’ágã pe’ẽvagué guarã, ojehechauka.'
'eu te amo' → '“Ojepe'a iporãiterei”.'
'obrigado' → 'I porã.'
'casa bonita' → 'Ko palanka oreko peteĩ mba’e iporãvéva oguereko peteĩva arýva ha ndaha’éi guarýpe, ko palanka iporãvéva.'

🌄 TESTANDO GUARANI -> PORTUGUÊS:
'Aiko porã' → 'Aiko ha’e peteĩ mba’e iñambuéva oñembohetavo haguã ñemoĩva umi mba’e iñambuéva.'
'Aguyje' → 'Ojejapo'
'Sy' → 'Sy'
'Kuarahy' → 'Kuarahy'

❓ Quer testar no modo interativo? (s/n): s
🔄 Carregando modelo treinado...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Modelo carregado no device: cuda

🎯 MODO INTERATIVO
Digite 'sair' para terminar
Digite 'pt:' + frase para traduzir para Guarani
Digite 'gn:' + frase para traduzir para Português

💬 Digite sua frase: pt
⚠️ Use o formato: 'pt:frase' ou 'gn:frase'

💬 Digite sua frase: Vamos comer
⚠️ Use o formato: 'pt:frase' ou 'gn:frase'

💬 Digite sua frase: pt:Vamos comer
🇧🇷 → 🇵🇾: 'Ahechaiterei, ojejuhúva.'
